In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('Data/winequality-red-cleaned.csv', sep=',')

In [10]:
df.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol,quality,category
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777,5,Bad
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777,5,Bad
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777,6,Good
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246,5,Bad


In [11]:
y = pd.DataFrame([0. if item == 'Good' else 1. for item in df['category']])
y = [0. if item == 'Good' else 1. for item in df['category']]
y[:5]

[1.0, 1.0, 1.0, 0.0, 1.0]

In [12]:
features_df = df.drop(['quality', 'category'], axis=1)

In [13]:
features_df.head()

,volatile acidity,residual sugar,chlorides,total sulfur dioxide,pH,sulphates,alcohol
0,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246
1,1.967442,0.043416,0.223875,0.624363,-0.719933,0.128950,-0.584777
2,1.297065,-0.169427,0.096353,0.229047,-0.331177,-0.048089,-0.584777
3,-1.384443,-0.453218,-0.264960,0.411500,-0.979104,-0.461180,-0.584777
4,0.961877,-0.453218,-0.243707,-0.379133,1.288643,-0.579207,-0.960246


In [14]:
import torch

In [15]:
# x = torch.from_numpy(features_df.as_matrix())

In [16]:
# y = torch.from_numpy(np.array(y))

In [17]:
# TODO: pip install sklearn
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_df, y, test_size=0.2, random_state=42)
# X_train = features_df
# y_train = np.array(y)

/Users/kendall/Dropbox/Current_Projects/python/pytorch_tutorial/.direnv/python-3.5.2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [33]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.activation import Softmax

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 2)

    def forward(self, x):
        x = F.softmax(self.fc1(x))
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()

In [39]:
learning_rate = 0.05
batch_size = 200
epochs = 1000
epochs_to_print = epochs / 10

In [35]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [58]:
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix, accuracy_score

for epoch in range(epochs): # loop over the dataset multiple times
    
    running_loss = 0.0
    for start, end in zip(range(0, len(X_train), batch_size), 
                          range(batch_size, len(X_train), batch_size)):
        # get the inputs
        
        inputs = torch.from_numpy(X_train[start:end].as_matrix())
        inputs = inputs.float()
        labels = torch.Tensor(y_train[start:end])
        labels = labels.long()

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()        
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        if epoch % epochs_to_print == 0:
            print(loss.data[0])
            running_loss = 0.0
            test_outputs = net(Variable(torch.from_numpy(X_test.as_matrix()).float()))
            _, predicted = torch.max(test_outputs.data, 1)
            accuracy = accuracy_score(y_test, predicted.numpy())
            print('%d loss: %.3f, accuracy: %.3f' % (epoch+1, running_loss / epochs_to_print,
                  accuracy))

print('Finished Training')

0.6424751281738281
1 loss: 0.000, accuracy: 0.728
0.628253698348999
1 loss: 0.000, accuracy: 0.728
0.641298234462738
1 loss: 0.000, accuracy: 0.734
0.6064736843109131
1 loss: 0.000, accuracy: 0.738
0.6074028015136719
1 loss: 0.000, accuracy: 0.738
0.5785908699035645
1 loss: 0.000, accuracy: 0.734
0.597084105014801
101 loss: 0.000, accuracy: 0.741
0.5555587410926819
101 loss: 0.000, accuracy: 0.741
0.6128346920013428
101 loss: 0.000, accuracy: 0.741
0.5328510999679565
101 loss: 0.000, accuracy: 0.741
0.5667860507965088
101 loss: 0.000, accuracy: 0.741
0.48340311646461487
101 loss: 0.000, accuracy: 0.741
0.5963595509529114
201 loss: 0.000, accuracy: 0.741
0.5549209117889404
201 loss: 0.000, accuracy: 0.741
0.6137063503265381
201 loss: 0.000, accuracy: 0.741
0.5336371064186096
201 loss: 0.000, accuracy: 0.741
0.5666817426681519
201 loss: 0.000, accuracy: 0.741
0.48055553436279297
201 loss: 0.000, accuracy: 0.741
0.5960546135902405
301 loss: 0.000, accuracy: 0.738
0.5547058582305908
301 lo

In [63]:
from sklearn.metrics import confusion_matrix
outputs = net(Variable(torch.from_numpy(X_test.as_matrix()).float()))
_, predicted = torch.max(outputs.data, 1)
confusion = confusion_matrix(y_test, predicted.numpy())
accuracy = accuracy_score(y_test, predicted.numpy())


In [64]:
confusion

array([[135,  44],
       [ 39, 102]])

In [65]:
accuracy

0.74062499999999998